# header

In [30]:
import numpy as np
import pandas as pd
import shutil

import os

update = False

In [31]:
from src.utils import *

DATA_PATH = './data'
        
def remove_space(sequence_str):
    return sequence_str.replace(' ', '')

In [ ]:
[name for name in all_proteins if ]

# Mauli's data

In [3]:
MAULI_PATH = f'{DATA_PATH}/_OGP_master/all_sites.csv'
RAW_COLUMNS = ['protein', 'positive_site', 'sequence']

def load_mauli(disp='all'):
    data = load_data(MAULI_PATH, columns=RAW_COLUMNS)
    data['sequence'] = data.sequence.apply(remove_space)
    
    print(f"data shape: {data.shape}")
    if disp == 'all':
        display(data)
        
    else:
        display(data.head(disp))
    
    return data


mauli_sites = load_mauli(disp=10)


data shape: (535, 3)


,protein,positive_site,sequence
0,A2AGT5,568,MGDDSEWLKLPVDQKCEHKLWKARLSGYEEALKIFQKIKDEKSPEW...
1,A2AHJ4,264,MAAAPTQIEAELYYLIARFLQSGPCNKSAQVLVQELEEHQLIPRRL...
2,A2AHJ4,279,MAAAPTQIEAELYYLIARFLQSGPCNKSAQVLVQELEEHQLIPRRL...
3,A2AKB9,87,MFPFGPHSPGGDETAGAEEPPPLGGPAAASRPPSPAPRPASPQRGA...
4,A2AKB9,99,MFPFGPHSPGGDETAGAEEPPPLGGPAAASRPPSPAPRPASPQRGA...
5,A2AKB9,100,MFPFGPHSPGGDETAGAEEPPPLGGPAAASRPPSPAPRPASPQRGA...
6,A2AKB9,364,MFPFGPHSPGGDETAGAEEPPPLGGPAAASRPPSPAPRPASPQRGA...
7,A2AQ25,357,MEESEGQKCEPNLPPSGDSRQMPQQGRSNLHVTSQEDAACRRPRER...
8,A6NKT7,919,MSCSKAYGERYVASVQGSAPSPRKKSTRGFYFAKLYYEAKEYDLAK...
9,E9K9Z1,39,MSKLVKTLTISEISKAQNNGGKPAWCWYTLAMCGAGYDSGTCDYMY...


In [4]:
path_n_name = f'{DATA_PATH}/mauli_sites.csv'

GROUP_COLUMNS = ['protein', 'positive_sites', 'sequence']
def group_sites(dataframe, disp='all'):
    protein_list = list(dataframe.protein.unique())
    
    dataset = pd.DataFrame(columns = GROUP_COLUMNS)
    for name in protein_list:
        data = dataframe[dataframe['protein']==name]
        sites = list(data.positive_site.values)
        sequence = data.sequence.iloc[0]
        protein_sites = pd.DataFrame([[name, sites, sequence]], columns=GROUP_COLUMNS)
        dataset = pd.concat([dataset, protein_sites], axis=0)
        
    dataset.reset_index(drop=True, inplace=True)
    
    print(f"data shape: {dataset.shape}")
    if disp == 'all':
        display(dataset)
        
    else:
        display(dataset.head(disp))
        
    return dataset


mauli_group = group_sites(mauli_sites, disp=10)
save_data(mauli_group, path_n_name, update=update)
mauli_proteins = list(mauli_group.protein.unique())
print(f'check protein # in Mauli: {len(mauli_proteins)}')


data shape: (273, 3)


,protein,positive_sites,sequence
0,A2AGT5,[568],MGDDSEWLKLPVDQKCEHKLWKARLSGYEEALKIFQKIKDEKSPEW...
1,A2AHJ4,"[264, 279]",MAAAPTQIEAELYYLIARFLQSGPCNKSAQVLVQELEEHQLIPRRL...
2,A2AKB9,"[87, 99, 100, 364]",MFPFGPHSPGGDETAGAEEPPPLGGPAAASRPPSPAPRPASPQRGA...
3,A2AQ25,[357],MEESEGQKCEPNLPPSGDSRQMPQQGRSNLHVTSQEDAACRRPRER...
4,A6NKT7,[919],MSCSKAYGERYVASVQGSAPSPRKKSTRGFYFAKLYYEAKEYDLAK...
5,E9K9Z1,[39],MSKLVKTLTISEISKAQNNGGKPAWCWYTLAMCGAGYDSGTCDYMY...
6,E9Q1P8,[208],MAAAVAVAAASRRQSCYLCDLPRMPWAMIWDFTEPVCRGCVNYEGA...
7,E9Q5G3,[720],MKSAKAKTVRKPVIKKGSQTNLKDPVGVYCRVRPLSFPDQECCVEV...
8,F6T0L5,[190],SQCLGREDLGHLCVWNDGCRLRGPLQPLPGTCPAPEIRAIEPLSGP...
9,F6TYF8,"[147, 148]",NSKVTVPHMIKSDARLHKDDTDICFSKTLNSCKVPQIRYASVERLL...


check protein # in Mauli: 273


# GlcNAcome data

In [5]:
GLCNACOME_PATH = f'{DATA_PATH}/_all_sites_oglcnac.csv'
GLCNACOME_COLUMNS = ['UniprotKB ID', 'organism', 'oglcnacscore', 'oglcnac sites', 'sequence']

def load_glcnacome(target_proteins=[], disp='all'):
    database = load_data(GLCNACOME_PATH)
    database_proteins = list(database['UniprotKB ID'].values)
    
    not_exist= []
    dataset = pd.DataFrame(columns=GLCNACOME_COLUMNS)
    i=1
    for protein in target_proteins:
        if protein in database_proteins:
            data = database.loc[database['UniprotKB ID']==protein][GLCNACOME_COLUMNS]
            dataset = pd.concat([dataset, data])
        
        elif protein[:6] in database_proteins and len(protein) > 6:
            data = database.loc[database['UniprotKB ID']==protein[:6]][GLCNACOME_COLUMNS]
            data['UniprotKB ID'] = protein
            dataset = pd.concat([dataset, data])
            
        else:
            not_exist.append(protein)
            print(f"{i:>2}. {protein} does not exist in Mauli's protein entries")
            i += 1
            
    dataset['sequence'] = dataset.sequence.apply(remove_space)
            
    print(f"data shape: {dataset.shape}")
    if disp == 'all':
        display(dataset)
        
    else:
        display(dataset.head(disp))
        
    return dataset, not_exist


glcnac_sites, na_proteins = load_glcnacome(target_proteins=mauli_proteins, disp=5)
glcnac_proteins = list(glcnac_sites['UniprotKB ID'].unique())
print(f'check protein # in GlcNAcome: {len(glcnac_proteins)}')
print(f"{len(na_proteins)} proteins in GlcNAc do not exist in Mauli's proteins")

 1. E9K9Z1 does not exist in Mauli's protein entries
 2. F6TYF8 does not exist in Mauli's protein entries
 3. O08984 does not exist in Mauli's protein entries
 4. O95832 does not exist in Mauli's protein entries
 5. P04799 does not exist in Mauli's protein entries
 6. P05451 does not exist in Mauli's protein entries
 7. P07756 does not exist in Mauli's protein entries
 8. P10688 does not exist in Mauli's protein entries
 9. P12729 does not exist in Mauli's protein entries
10. P12939 does not exist in Mauli's protein entries
11. P20023 does not exist in Mauli's protein entries
12. P24329 does not exist in Mauli's protein entries
13. P29147 does not exist in Mauli's protein entries
14. P52285 does not exist in Mauli's protein entries
15. P54274 does not exist in Mauli's protein entries
16. P54939 does not exist in Mauli's protein entries
17. Q01814 does not exist in Mauli's protein entries
18. Q02156 does not exist in Mauli's protein entries
19. Q05655 does not exist in Mauli's protein e

,UniprotKB ID,organism,oglcnacscore,oglcnac sites,sequence
497,A2AGT5,Mus musculus,21.955739,S568,MGDDSEWLKLPVDQKCEHKLWKARLSGYEEALKIFQKIKDEKSPEW...
502,A2AHJ4,Mus musculus,12.197459,S264;T279,MAAAPTQIEAELYYLIARFLQSGPCNKSAQVLVQELEEHQLIPRRL...
513,A2AKB9,Mus musculus,12.197459,T87;S99;S100;S364,MFPFGPHSPGGDETAGAEEPPPLGGPAAASRPPSPAPRPASPQRGA...
529,A2AQ25,Mus musculus,37.299298,S357,MEESEGQKCEPNLPPSGDSRQMPQQGRSNLHVTSQEDAACRRPRER...
643,A6NKT7,Homo sapiens,9.062861,S919,MSCSKAYGERYVASVQGSAPSPRKKSTRGFYFAKLYYEAKEYDLAK...


check protein # in GlcNAcome: 248
25 proteins in GlcNAc do not exist in Mauli's proteins


In [6]:
path_n_name = f'{DATA_PATH}/glcnacome_sites.csv'

def process_glcnacome(dataframe, disp='all'):
    
    def split_sites(sites_str):
        return sites_str.split(';') if isinstance(sites_str, str) else []

    def get_int(sites_list):
        return [int(x[1:]) if len(x) < 8 else int(x.split(' ')[0][1:]) for x in sites_list] 
        # Example format '123 (P10636-8)'
        
    dataframe = dataframe.rename(columns={'UniprotKB ID':'protein', 'oglcnac sites':'positive_sites'})
    dataframe['positive_sites'] = dataframe['positive_sites'].apply(split_sites).apply(get_int)
    
    print(f"data shape: {dataframe.shape}")
    if disp == 'all':
        display(dataframe)
        
    else:
        display(dataframe.head(disp))
        
    return dataframe


glcnac_group = process_glcnacome(dataframe=glcnac_sites, disp=5)
save_data(glcnac_group, path_n_name, update=update)
save_data(glcnac_group, path_n_name[:-3]+'pkl', update=update)


data shape: (248, 5)


,protein,organism,oglcnacscore,positive_sites,sequence
497,A2AGT5,Mus musculus,21.955739,[568],MGDDSEWLKLPVDQKCEHKLWKARLSGYEEALKIFQKIKDEKSPEW...
502,A2AHJ4,Mus musculus,12.197459,"[264, 279]",MAAAPTQIEAELYYLIARFLQSGPCNKSAQVLVQELEEHQLIPRRL...
513,A2AKB9,Mus musculus,12.197459,"[87, 99, 100, 364]",MFPFGPHSPGGDETAGAEEPPPLGGPAAASRPPSPAPRPASPQRGA...
529,A2AQ25,Mus musculus,37.299298,[357],MEESEGQKCEPNLPPSGDSRQMPQQGRSNLHVTSQEDAACRRPRER...
643,A6NKT7,Homo sapiens,9.062861,[919],MSCSKAYGERYVASVQGSAPSPRKKSTRGFYFAKLYYEAKEYDLAK...


# Concatenate datasets: positivity, flexibility, and secondary structure

In [7]:
from src.protein_properties import Properties

PROPERTIES = Properties(DATA_PATH, protein_name=glcnac_proteins[0])
display(PROPERTIES.all_properties())

PROPERTIES = Properties(DATA_PATH, protein_name=glcnac_proteins[1])
display(PROPERTIES.all_properties())

A2AGT5   do not match between length in positivity, flexibility, and secondary_structure


[]

,protein,#,SEQ,positivity,flexibility,SS,ASA,Phi,Psi,Theta(i-1=>i+1),Tau(i-2=>i+2),HSE_alpha_up,HSE_alpha_down,P(C),P(H),P(E)
A2AHJ4,,,,,,,,,,,,,,,,
0,A2AHJ4,1,M,0,0.511,C,151.0,-94.2,133.0,115.1,155.6,3.9,5.6,0.991,0.007,0.002
1,A2AHJ4,2,A,0,0.507,C,91.5,-88.5,136.1,114.5,-132.9,3.7,7.4,0.983,0.010,0.007
2,A2AHJ4,3,A,0,0.536,C,79.9,-89.2,133.8,113.6,-128.4,5.2,11.5,0.963,0.019,0.018
3,A2AHJ4,4,A,0,0.586,C,65.3,-77.9,140.0,112.8,-108.1,7.9,13.5,0.959,0.037,0.003
4,A2AHJ4,5,P,0,0.625,C,71.5,-67.7,142.6,105.7,-106.1,10.1,13.8,0.708,0.290,0.002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,A2AHJ4,1795,M,0,0.621,C,115.8,-88.8,5.3,100.3,51.7,4.5,12.6,0.883,0.096,0.021
1795,A2AHJ4,1796,G,0,0.582,C,51.7,82.7,2.3,104.8,-83.4,3.7,14.9,0.966,0.016,0.019
1796,A2AHJ4,1797,W,0,0.535,C,127.8,-95.6,123.9,113.9,166.1,9.7,9.4,0.927,0.041,0.032


# Mauli's features

In [8]:
from src.mauli import *

## each protein

In [9]:
MAULI_DATASET_PATH = f'{DATA_PATH}/mauli_dataset'
create_dir(MAULI_DATASET_PATH)

SEQUENCE_WINDOW = (-10,10)
SIDE_WINDOW = (-1, 5)
ALIPHATIC_WINDOW = (-3, -1)
POSITIVE_CHARGE_WINDOW = (-7, -5)
PROLINE_AFTER = 1
    
def mauli_features(properties:object):
    dataset = properties.all_properties()
    
    if len(dataset):
        sequence = dataset['SEQ'].sum()
        dataset['window'] = dataset['#'].apply(lambda x: make_window(sequence=sequence, index=x, window=SEQUENCE_WINDOW))
        dataset['marker'] = dataset['#'].apply(lambda x: make_window(sequence=sequence, index=x, window=SEQUENCE_WINDOW, marking=True))
    
        for side in range(SIDE_WINDOW[0], SIDE_WINDOW[1] + 1):
            if side != 0:
                dataset[f'side_{side}'] = dataset.marker.apply(lambda x: side_chain(x, side=side))

        dataset[f'nAli{ALIPHATIC_WINDOW}'] = dataset.apply(lambda x: count_nonpolar_aliphatic(make_window(sequence, x['#'], ALIPHATIC_WINDOW)), axis=1)
        dataset[f'nPos{POSITIVE_CHARGE_WINDOW}'] = dataset.apply(lambda x: count_positively_charged(make_window(sequence, x['#'], POSITIVE_CHARGE_WINDOW)), axis=1)
        dataset['nS/nT'] = dataset.window.apply(count_SorT)
        dataset[f'is_proline({PROLINE_AFTER})'] = dataset.marker.apply(lambda x: is_proline_after(x, after=PROLINE_AFTER))
        dataset['phi_psi'] = dataset.apply(lambda x: angle_type(x['Phi'], x['Psi']), axis=1)
        
        return dataset
    
    else:
        return []

i = 0
pass_list = []
for protein_name in glcnac_proteins:
    path_n_name = f'{MAULI_DATASET_PATH}/{protein_name}.pkl'
    mauli_dataset = mauli_features(Properties(DATA_PATH, protein_name))
    
    if len(mauli_dataset):
        save_data(mauli_dataset, path_n_name, update=update)
        
        if i == 0:
            save_data(mauli_dataset, f'{path_n_name[:-3]}.csv', update=update)
            i += 1
        
    else:
        pass_list.append(protein_name)
        pass
    
print(f"{len(pass_list)} proteins were added to pass_list")

A2AGT5   do not match between length in positivity, flexibility, and secondary_structure
F6T0L5   do not match between length in positivity, flexibility, and secondary_structure
P19246   do not match between length in positivity, flexibility, and secondary_structure
P24622_2 do not match between length in positivity, flexibility, and secondary_structure
P48614_2 do not match between length in positivity, flexibility, and secondary_structure
Q01705_2 do not match between length in positivity, flexibility, and secondary_structure
Q03173_4 do not match between length in positivity, flexibility, and secondary_structure
Q04690_4 do not match between length in positivity, flexibility, and secondary_structure
Q4U2R1_2 do not match between length in positivity, flexibility, and secondary_structure
Q62381_2 do not match between length in positivity, flexibility, and secondary_structure
Q62417_5 do not match between length in positivity, flexibility, and secondary_structure
Q69ZI1_3 do not match

## all proteins

In [10]:
all_proteins = [x for x in glcnac_proteins if x not in pass_list]
path_n_name = f'{DATA_PATH}/mauli_feature_for_all_AA.pkl'

DTYPES = {'positivity': 'int64', 
          f'nAli{ALIPHATIC_WINDOW}': 'int64',
          f'nPos{POSITIVE_CHARGE_WINDOW}': 'int64',
          'nS/nT': 'int64',
          f'is_proline({PROLINE_AFTER})': 'int64'}

if not check_file(path_n_name, update=update):
    mauli_feature_for_all_AA = pd.concat([pd.read_pickle(f'{MAULI_DATASET_PATH}/{name}.pkl') for name in all_proteins], axis=0)
    
    mauli_feature_for_all_AA = mauli_feature_for_all_AA.astype(DTYPES)
    mauli_feature_for_all_AA.reset_index(drop=True, inplace=True)
    save_data(mauli_feature_for_all_AA, path_n_name, update=update)
    
else:
    mauli_feature_for_all_AA = load_data(path_n_name)

print(f'data shape: {mauli_feature_for_all_AA.shape}')
display(mauli_feature_for_all_AA.head(5))
print(f'check protein # in Mauli features: {len(all_proteins)}')

data shape: (221789, 29)


,protein,#,SEQ,positivity,flexibility,SS,ASA,Phi,Psi,Theta(i-1=>i+1),...,side_1,side_2,side_3,side_4,side_5,"nAli(-3, -1)","nPos(-7, -5)",nS/nT,is_proline(1),phi_psi
0,A2AHJ4,1,M,0,0.511,C,151.0,-94.2,133.0,115.1,...,very_small,very_small,very_small,pro,small,0,0,1,0,alpha
1,A2AHJ4,2,A,0,0.507,C,91.5,-88.5,136.1,114.5,...,very_small,very_small,pro,small,normal,0,0,1,0,alpha
2,A2AHJ4,3,A,0,0.536,C,79.9,-89.2,133.8,113.6,...,very_small,pro,small,normal,small,1,0,1,0,alpha
3,A2AHJ4,4,A,0,0.586,C,65.3,-77.9,140.0,112.8,...,pro,small,normal,small,normal,2,0,1,1,alpha
4,A2AHJ4,5,P,0,0.625,C,71.5,-67.7,142.6,105.7,...,small,normal,small,normal,very_small,3,0,1,0,alpha


check protein # in Mauli features: 230


## serine or threonine (for machine learning model)

In [11]:
mauli_feature_for_ml = mauli_feature_for_all_AA.loc[(mauli_feature_for_all_AA['SEQ']=='S') | (mauli_feature_for_all_AA['SEQ']=='T')]
mauli_feature_for_ml.reset_index(drop=True, inplace=True)

print(f'data shape: {mauli_feature_for_ml.shape}')
display(mauli_feature_for_ml.head(5))

data shape: (36384, 29)


,protein,#,SEQ,positivity,flexibility,SS,ASA,Phi,Psi,Theta(i-1=>i+1),...,side_1,side_2,side_3,side_4,side_5,"nAli(-3, -1)","nPos(-7, -5)",nS/nT,is_proline(1),phi_psi
0,A2AHJ4,6,T,0,0.675,C,80.6,-78.9,-11.1,102.8,...,normal,small,normal,very_small,normal,3,0,1,0,beta
1,A2AHJ4,22,S,0,0.717,C,60.8,-97.4,-6.6,102.2,...,gly,pro,small,normal,long,1,0,2,0,beta
2,A2AHJ4,28,S,0,0.771,H,5.4,-65.8,-43.4,94.1,...,very_small,normal,very_small,small,very_small,0,0,2,0,beta
3,A2AHJ4,56,S,0,0.785,C,36.4,-108.0,166.7,129.9,...,cycle,normal,normal,small,very_small,0,1,1,0,alpha
4,A2AHJ4,90,S,0,0.757,C,66.2,-83.6,-0.5,98.6,...,very_small,pro,gly,very_small,normal,2,1,2,0,beta


In [12]:
mauli_feature_for_ml.dtypes

protein             object
#                    int64
SEQ                 object
positivity           int64
flexibility        float64
SS                  object
ASA                float64
Phi                float64
Psi                float64
Theta(i-1=>i+1)    float64
Tau(i-2=>i+2)      float64
HSE_alpha_up       float64
HSE_alpha_down     float64
P(C)               float64
P(H)               float64
P(E)               float64
window              object
marker              object
side_-1             object
side_1              object
side_2              object
side_3              object
side_4              object
side_5              object
nAli(-3, -1)         int64
nPos(-7, -5)         int64
nS/nT                int64
is_proline(1)        int64
phi_psi             object
dtype: object

In [13]:
mauli_feature_for_ml.describe()

,#,positivity,flexibility,ASA,Phi,Psi,Theta(i-1=>i+1),Tau(i-2=>i+2),HSE_alpha_up,HSE_alpha_down,P(C),P(H),P(E),"nAli(-3, -1)","nPos(-7, -5)",nS/nT,is_proline(1)
count,36384.000000,36384.000000,36384.000000,36384.000000,36384.000000,36384.000000,36384.000000,36384.000000,36384.000000,36384.000000,36384.000000,36384.000000,36384.000000,36384.000000,36384.000000,36384.000000,36384.000000
mean,919.939836,0.074428,0.701680,58.562753,-87.843923,64.535686,108.521221,-21.194074,9.631517,16.922246,0.698350,0.189607,0.112036,1.000385,0.384922,5.026715,0.096141
std,865.538257,0.262470,0.109638,23.911847,16.145003,72.580283,9.764710,119.302201,5.511618,5.157718,0.298889,0.288419,0.186790,0.839970,0.604706,2.567014,0.294789
min,2.000000,0.000000,0.226000,0.100000,-169.200000,-178.600000,86.900000,-180.000000,0.400000,2.800000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,285.000000,0.000000,0.628000,42.775000,-96.300000,-10.200000,100.800000,-144.200000,5.500000,13.300000,0.589000,0.015000,0.013000,0.000000,0.000000,3.000000,0.000000
50%,628.000000,0.000000,0.705000,61.300000,-87.500000,87.200000,108.600000,49.200000,8.700000,16.500000,0.815000,0.061000,0.045000,1.000000,0.000000,5.000000,0.000000
75%,1333.000000,0.000000,0.780000,75.700000,-78.400000,134.700000,116.000000,76.000000,12.400000,20.100000,0.918000,0.197000,0.116000,2.000000,1.000000,7.000000,0.000000
max,5067.000000,1.000000,1.207000,123.100000,133.800000,179.400000,147.800000,180.000000,30.100000,32.400000,1.000000,1.000000,0.999000,3.000000,3.000000,21.000000,1.000000


# Data augmentation

## check, move, and copy data

In [14]:
_KRISHNA_PATH = f'{DATA_PATH}/_from_Krishna'
KRISHNA_PATH  = f'{DATA_PATH}/from_Krishna'
create_dir(KRISHNA_PATH)

In [15]:
import shutil
FEATURE_FILE = 'OUTPUT-WINDOW-all-combine.csv'

def check_and_copy(name):
    protein_dir = f"{KRISHNA_PATH}/{name}"
    check_dir = f"{protein_dir}/all_AA/all_AA_new_feature_window"
    
    if exists(protein_dir) and not exists(check_dir):
        os.makedirs(check_dir)
        shutil.copyfile(f'{protein_dir}/{feature_file}', 
                        f'{check_dir}/{feature_file}')
    
def copy_protein(name, copy_path, name_2=None):
    check_and_copy(name)
    from_file = f'{KRISHNA_PATH}/{name}/all_AA/all_AA_new_feature_window/{FEATURE_FILE}'
    
    if name_2:
        to_file = f'{copy_path}/{name_2}.csv'
    else:
        to_file = f'{copy_path}/{name}.csv'
    
    if not exists(to_file):
        shutil.copyfile(from_file, to_file)

In [16]:
check_proteins = [name for name in all_proteins if exists(f"{_KRISHNA_PATH}/{name}")]

for name in check_proteins:
    protein_dir = f'{_KRISHNA_PATH}/{name}'
    if not exists(f"{protein_dir}/all_AA"):
        create_dir(f"{protein_dir}/all_AA")
        
    if not exists(f"{protein_dir}/all_AA/all_AA_new_feature_window"):
        create_dir(f"{protein_dir}/all_AA/all_AA_new_feature_window")

    if update or not exists(f'{protein_dir}/all_AA/all_AA_new_feature_window/{FEATURE_FILE}'): 
        shutil.copyfile(f'{protein_dir}/all_AA_new_feature_window/{FEATURE_FILE}', 
                        f'{protein_dir}/all_AA/all_AA_new_feature_window/{FEATURE_FILE}') 


In [21]:
convert_proteins = {
    'A0A024RAY2':'P05783',    
    'P0CG62':'P0CG49',        
    'P63249':'P63248',        
    'Q4R561_P60710':'P60710', 
    'Q9WVB1':'P35279',        
}

for key, value in convert_proteins.items():
    from_dir = f'{_KRISHNA_PATH}/{key}/all_AA/all_AA_new_feature_window/{FEATURE_FILE}'
    to_dir   = f'{KRISHNA_PATH}/{value}.csv'
    
    if not exists(to_dir) or update:
        shutil.copyfile(from_dir, to_dir)

In [22]:
_krishna_proteins = [name for name in all_proteins if exists(f"{_KRISHNA_PATH}/{name}")]

for name in _krishna_proteins:
    from_dir = f'{_KRISHNA_PATH}/{name}/all_AA/all_AA_new_feature_window/{FEATURE_FILE}'
    to_dir   = f'{KRISHNA_PATH}/{name}.csv'
    shutil.copyfile(from_dir, to_dir)

In [23]:
FOLDER_KRISHNA = [x for x in os.listdir(_KRISHNA_PATH) if x[-3:]!='csv']
NOT_IN_AUGMENTED = []
for name in FOLDER_KRISHNA:
    if not exists(f'{KRISHNA_PATH}/{name}.csv') and name not in convert_proteins.keys(): 
        if name not in pass_list:
            NOT_IN_AUGMENTED.append(name)
        
print(f'{len(NOT_IN_AUGMENTED)} proteins exist in Krishna path but not in our dataset') 
print('due to that they are not in GlcNAcome database')
NOT_IN_AUGMENTED

12 proteins exist in Krishna path but not in our dataset
due to that they are not in GlcNAcome database


['24622_2',
 'Q9WVB1_P35279',
 'P68406_P24622_2',
 'P63249_P63248',
 'P04799',
 'P05451',
 'P07756',
 'P0CG62_P0CG49',
 'E9K9Z1',
 'O08984',
 'A0A024RAY2_P05783',
 'A2ABU4']

In [24]:
krishna_proteins = [x[:-4] for x in os.listdir(KRISHNA_PATH) if x[-3:]=='csv']
print('total number of krishna proteins:', len(krishna_proteins))

total number of krishna proteins: 107


## data augmentation

In [25]:
def z_feature(idx, sequence):
    return idx / len(sequence)

In [26]:
pd.read_csv(f'{_KRISHNA_PATH}/features_all_names_new_window.csv', header=None)

,0,1,2,3,4,5,6,7,8,9,...,494,495,496,497,498,499,500,501,502,503
0,resid,number_hydrophobic_0A,number_hydrophilic_0A,number_polar_0A,number_aromatic_0A,number_aliphatic_0A,number_charged_0A,number_positive_0A,number_negative_0A,number_gly_0A,...,sasa_tyr_25A,sasa_back_25A,sasa_side_25A,sasa_target_ser_thr_25A,net_charge_all_25A,net_charge_backbone_25A,net_charge_sidechain_25A,net_charge_all_exposed_25A,net_charge_backbone_exposed_25A,net_charge_sidechain_exposed_25A


In [27]:
augmented_features = pd.read_csv(f'{_KRISHNA_PATH}/features_all_names_new_window.csv', header=None).values.squeeze()

AUGMENT_PATH = f'{DATA_PATH}/augmented_feature'
create_dir(AUGMENT_PATH)
    
null_proteins = []
i=0
for name in krishna_proteins:
    path_n_name = f'{AUGMENT_PATH}/{name}.pkl'
    if not check_file(path_n_name, update=update):
        alpha_file = pd.read_csv(f'{KRISHNA_PATH}/{name}.csv', sep=' ', header=None, names=augmented_features)
        alpha_file.sort_values(by='resid').reset_index(drop=True, inplace=True)

        mauli_file = pd.read_pickle(f'{MAULI_DATASET_PATH}/{name}.pkl')

        assert len(mauli_file) == len(alpha_file), f'{name} do not match between length in Mauli and AlphaFold'
        augmented_file = pd.concat([mauli_file, alpha_file.iloc[:,1:]], axis=1)
        augmented_file['z'] = augmented_file['#'].apply(lambda x: x/len(augmented_file.SEQ.sum()))
        
        if augmented_file.isna().any().any():
            null_proteins.append(name)
        else:
            augmented_file.to_pickle(path_n_name)
        
            if i == 0:
                augmented_file.to_csv(path_n_name[:-3]+'csv')
                i += 1
        
    else:
        augmented_file = pd.read_pickle(path_n_name)
        
augmented_file

,protein,#,SEQ,positivity,flexibility,SS,ASA,Phi,Psi,Theta(i-1=>i+1),...,sasa_back_25A,sasa_side_25A,sasa_target_ser_thr_25A,net_charge_all_25A,net_charge_backbone_25A,net_charge_sidechain_25A,net_charge_all_exposed_25A,net_charge_backbone_exposed_25A,net_charge_sidechain_exposed_25A,z
0,Q9QXD8,1,M,0,0.519,C,128.1,-93.2,131.2,114.1,...,348.979999,4174.980011,4848.480009,-0.999999,-24.14,-1.55,-7.739999,-17.87,0.080001,0.001497
1,Q9QXD8,2,D,0,0.536,C,105.6,-84.8,128.8,111.6,...,296.340000,3972.430003,4549.140002,1.000001,-20.36,0.27,-9.519999,-19.24,1.750000,0.002994
2,Q9QXD8,3,K,0,0.570,C,119.0,-75.3,-17.8,98.6,...,228.099999,4451.749999,4926.199996,-0.999999,-19.07,-1.82,-12.069999,-19.10,0.390000,0.004491
3,Q9QXD8,4,Y,0,0.633,C,107.2,-75.9,-19.6,97.7,...,329.139999,5100.060004,5738.060002,-0.999999,-23.96,-1.73,-13.039999,-22.61,0.880001,0.005988
4,Q9QXD8,5,D,0,0.676,C,92.0,-71.0,-29.6,96.1,...,434.140000,5604.809998,6425.859997,-0.999999,-29.25,-1.64,-14.319999,-26.51,1.100001,0.007485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,Q9QXD8,664,H,0,0.507,C,98.2,-99.2,118.3,113.1,...,580.020003,2971.479993,3915.879995,1.000000,-13.77,2.90,-1.200000,-13.71,3.830000,0.994012
664,Q9QXD8,665,Q,0,0.515,C,123.8,-83.9,7.0,102.5,...,310.870002,2029.939994,2538.149994,1.000000,-7.90,2.63,-1.370000,-9.42,3.330000,0.995509
665,Q9QXD8,666,H,0,0.481,C,86.3,-96.0,126.0,114.2,...,316.090002,1865.939996,2392.239997,1.000000,-7.50,2.63,-0.350000,-8.91,3.620000,0.997006
666,Q9QXD8,667,H,0,0.458,C,119.5,-96.2,122.5,113.1,...,190.700000,1340.149999,1664.039999,2.000000,-5.01,3.54,0.670000,-6.97,4.170000,0.998503


In [28]:
augmented_proteins = [name for name in krishna_proteins if name not in null_proteins]
print('total number of augmented proteins:', len(augmented_proteins))
print('null proteins:', null_proteins)

total number of augmented proteins: 106
null proteins: ['P26039']


## Mauli's feature for Krishna's proteins

In [29]:
MAULI_AUGMENT_PATH = f'{DATA_PATH}/mauli_feature(augmented_proteins)'
create_dir(MAULI_AUGMENT_PATH)
    
for name in augmented_proteins:
    shutil.copyfile(f'{MAULI_DATASET_PATH}/{name}.pkl', 
                    f'{MAULI_AUGMENT_PATH}/{name}.pkl')